In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from IPython.display import display

from sklearn import metrics
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
sns.set()
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from xgboost import XGBRegressor
from sklearn.metrics import roc_auc_score

In [3]:
import os
PATH = os.getcwd();
PATH = PATH+'\\AV_Mckin\\ods\\';
PATH

'D:\\Github\\fastai\\courses\\ml1\\AV_Mckin\\ods\\'

In [57]:
df_train_org = pd.read_csv(f'{PATH}\\flight_delays_train.csv', low_memory=False)
df_test_org = pd.read_csv(f'{PATH}\\flight_delays_test.csv', low_memory=False)
df_raw = pd.read_csv(f'{PATH}\\impact_encoded_train.csv', low_memory=False)
df_test = pd.read_csv(f'{PATH}\\impact_encoded_test.csv', low_memory=False)

In [58]:
df_raw.head(2)

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,target,impact_encoded_Month,impact_encoded_DayofMonth,impact_encoded_DayOfWeek,impact_encoded_UniqueCarrier,impact_encoded_Origin,impact_encoded_Dest
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,0,0.202561,0.202928,0.191829,0.186803,0.258108,0.148708
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,0,0.155567,0.213503,0.176398,0.167930,0.178860,0.177618


In [8]:
from sklearn.model_selection import train_test_split

In [65]:
def preprocess(X):
    X["Flight"] = X["Origin"] + "-" + X["Dest"]
    X["Hour"] = X["DepTime"] // 100
    X['Mins'] = X['DepTime'] % 100
    X["Month"] = X["Month"].apply(lambda x: int(x.split('-')[1]))
    X["DayOfMonth"] = X["DayofMonth"].apply(lambda x: int(x.split('-')[1]))
    X = X.drop(["DayofMonth"], axis=1)
    X['DayOfWeek'] = X['DayOfWeek'].apply(lambda x: int(x.split('-')[1]))
    X['weekend'] = np.where(X['DayOfWeek']>=6,1,0)
    return X

In [66]:
df_raw  = preprocess(df_raw.copy())
df_test = preprocess(df_test.copy())

In [67]:
y = df_raw.target.values
df_raw.drop('target',axis=1,inplace=True)
categorical_features_indices = np.where(df_raw.dtypes == object)[0]
categorical_features_indices

array([ 3,  4,  5, 13], dtype=int64)

In [68]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(df_raw, y, train_size=0.8, random_state=1234);

In [69]:
#importing library and building model
from catboost import CatBoostClassifier
model=CatBoostClassifier(iterations=1000, depth=10, learning_rate=0.01, loss_function='CrossEntropy',\
                         )
model.fit(X_train, y_train,cat_features=categorical_features_indices,eval_set=(X_validation, y_validation))

0:	learn: 0.6877879	test: 0.6877784	best: 0.6877784 (0)	total: 207ms	remaining: 3m 27s
1:	learn: 0.6825222	test: 0.6824822	best: 0.6824822 (1)	total: 533ms	remaining: 4m 25s
2:	learn: 0.6771314	test: 0.6770637	best: 0.6770637 (2)	total: 820ms	remaining: 4m 32s
3:	learn: 0.6722187	test: 0.6720943	best: 0.6720943 (3)	total: 976ms	remaining: 4m 2s
4:	learn: 0.6671047	test: 0.6669208	best: 0.6669208 (4)	total: 1.4s	remaining: 4m 37s
5:	learn: 0.6621384	test: 0.6619802	best: 0.6619802 (5)	total: 1.67s	remaining: 4m 37s
6:	learn: 0.6575295	test: 0.6573748	best: 0.6573748 (6)	total: 1.78s	remaining: 4m 12s
7:	learn: 0.6530926	test: 0.6530158	best: 0.6530158 (7)	total: 2.19s	remaining: 4m 31s
8:	learn: 0.6486140	test: 0.6485535	best: 0.6485535 (8)	total: 2.56s	remaining: 4m 42s
9:	learn: 0.6440716	test: 0.6440460	best: 0.6440460 (9)	total: 2.93s	remaining: 4m 50s
10:	learn: 0.6396833	test: 0.6395909	best: 0.6395909 (10)	total: 3.13s	remaining: 4m 41s
11:	learn: 0.6351456	test: 0.6350678	best: 

93:	learn: 0.4753141	test: 0.4750579	best: 0.4750579 (93)	total: 25.6s	remaining: 4m 6s
94:	learn: 0.4746413	test: 0.4743904	best: 0.4743904 (94)	total: 25.7s	remaining: 4m 5s
95:	learn: 0.4740408	test: 0.4737976	best: 0.4737976 (95)	total: 25.9s	remaining: 4m 3s
96:	learn: 0.4733292	test: 0.4730864	best: 0.4730864 (96)	total: 26.2s	remaining: 4m 3s
97:	learn: 0.4726834	test: 0.4724513	best: 0.4724513 (97)	total: 26.4s	remaining: 4m 2s
98:	learn: 0.4719432	test: 0.4717310	best: 0.4717310 (98)	total: 26.8s	remaining: 4m 3s
99:	learn: 0.4711617	test: 0.4709654	best: 0.4709654 (99)	total: 27.1s	remaining: 4m 4s
100:	learn: 0.4706076	test: 0.4704319	best: 0.4704319 (100)	total: 27.6s	remaining: 4m 5s
101:	learn: 0.4700848	test: 0.4699148	best: 0.4699148 (101)	total: 27.7s	remaining: 4m 3s
102:	learn: 0.4695464	test: 0.4693823	best: 0.4693823 (102)	total: 27.9s	remaining: 4m 2s
103:	learn: 0.4689858	test: 0.4688493	best: 0.4688493 (103)	total: 28.3s	remaining: 4m 4s
104:	learn: 0.4683744	te

185:	learn: 0.4420779	test: 0.4427036	best: 0.4427036 (185)	total: 57.4s	remaining: 4m 11s
186:	learn: 0.4419770	test: 0.4426062	best: 0.4426062 (186)	total: 57.6s	remaining: 4m 10s
187:	learn: 0.4417953	test: 0.4424418	best: 0.4424418 (187)	total: 58.1s	remaining: 4m 10s
188:	learn: 0.4416095	test: 0.4422800	best: 0.4422800 (188)	total: 58.5s	remaining: 4m 11s
189:	learn: 0.4414157	test: 0.4420853	best: 0.4420853 (189)	total: 58.9s	remaining: 4m 10s
190:	learn: 0.4412573	test: 0.4419307	best: 0.4419307 (190)	total: 59.3s	remaining: 4m 11s
191:	learn: 0.4411220	test: 0.4418057	best: 0.4418057 (191)	total: 59.5s	remaining: 4m 10s
192:	learn: 0.4410290	test: 0.4417068	best: 0.4417068 (192)	total: 59.6s	remaining: 4m 9s
193:	learn: 0.4409378	test: 0.4416220	best: 0.4416220 (193)	total: 59.8s	remaining: 4m 8s
194:	learn: 0.4407424	test: 0.4414340	best: 0.4414340 (194)	total: 1m	remaining: 4m 8s
195:	learn: 0.4406122	test: 0.4413153	best: 0.4413153 (195)	total: 1m	remaining: 4m 7s
196:	lear

276:	learn: 0.4321911	test: 0.4339178	best: 0.4339178 (276)	total: 1m 31s	remaining: 3m 57s
277:	learn: 0.4321339	test: 0.4338693	best: 0.4338693 (277)	total: 1m 31s	remaining: 3m 57s
278:	learn: 0.4320924	test: 0.4338293	best: 0.4338293 (278)	total: 1m 31s	remaining: 3m 56s
279:	learn: 0.4319772	test: 0.4337691	best: 0.4337691 (279)	total: 1m 32s	remaining: 3m 57s
280:	learn: 0.4319226	test: 0.4337255	best: 0.4337255 (280)	total: 1m 32s	remaining: 3m 57s
281:	learn: 0.4318416	test: 0.4336523	best: 0.4336523 (281)	total: 1m 33s	remaining: 3m 57s
282:	learn: 0.4317427	test: 0.4335624	best: 0.4335624 (282)	total: 1m 33s	remaining: 3m 57s
283:	learn: 0.4316738	test: 0.4335074	best: 0.4335074 (283)	total: 1m 34s	remaining: 3m 57s
284:	learn: 0.4316119	test: 0.4334653	best: 0.4334653 (284)	total: 1m 34s	remaining: 3m 57s
285:	learn: 0.4315504	test: 0.4334112	best: 0.4334112 (285)	total: 1m 35s	remaining: 3m 57s
286:	learn: 0.4314923	test: 0.4333689	best: 0.4333689 (286)	total: 1m 35s	remain

366:	learn: 0.4269453	test: 0.4299276	best: 0.4299276 (366)	total: 2m 5s	remaining: 3m 37s
367:	learn: 0.4269166	test: 0.4299140	best: 0.4299140 (367)	total: 2m 6s	remaining: 3m 36s
368:	learn: 0.4268925	test: 0.4298984	best: 0.4298984 (368)	total: 2m 6s	remaining: 3m 36s
369:	learn: 0.4268365	test: 0.4298563	best: 0.4298563 (369)	total: 2m 6s	remaining: 3m 35s
370:	learn: 0.4267759	test: 0.4298314	best: 0.4298314 (370)	total: 2m 7s	remaining: 3m 35s
371:	learn: 0.4267432	test: 0.4297971	best: 0.4297971 (371)	total: 2m 7s	remaining: 3m 35s
372:	learn: 0.4266611	test: 0.4297470	best: 0.4297470 (372)	total: 2m 8s	remaining: 3m 35s
373:	learn: 0.4266507	test: 0.4297388	best: 0.4297388 (373)	total: 2m 8s	remaining: 3m 34s
374:	learn: 0.4266067	test: 0.4297230	best: 0.4297230 (374)	total: 2m 8s	remaining: 3m 34s
375:	learn: 0.4265460	test: 0.4296907	best: 0.4296907 (375)	total: 2m 8s	remaining: 3m 34s
376:	learn: 0.4265154	test: 0.4296655	best: 0.4296655 (376)	total: 2m 9s	remaining: 3m 33s

...

636:	learn: 0.4180245	test: 0.4246859	best: 0.4246859 (636)	total: 3m 49s	remaining: 2m 10s
637:	learn: 0.4180223	test: 0.4246837	best: 0.4246837 (637)	total: 3m 49s	remaining: 2m 10s
638:	learn: 0.4179902	test: 0.4246718	best: 0.4246718 (638)	total: 3m 50s	remaining: 2m 10s
639:	learn: 0.4179532	test: 0.4246548	best: 0.4246548 (639)	total: 3m 50s	remaining: 2m 9s
640:	learn: 0.4179375	test: 0.4246485	best: 0.4246485 (640)	total: 3m 51s	remaining: 2m 9s
641:	learn: 0.4178976	test: 0.4246352	best: 0.4246352 (641)	total: 3m 51s	remaining: 2m 9s
642:	learn: 0.4178602	test: 0.4246045	best: 0.4246045 (642)	total: 3m 51s	remaining: 2m 8s
643:	learn: 0.4178452	test: 0.4245955	best: 0.4245955 (643)	total: 3m 52s	remaining: 2m 8s
644:	learn: 0.4178391	test: 0.4245912	best: 0.4245912 (644)	total: 3m 52s	remaining: 2m 8s
645:	learn: 0.4178033	test: 0.4245694	best: 0.4245694 (645)	total: 3m 53s	remaining: 2m 7s
646:	learn: 0.4177770	test: 0.4245543	best: 0.4245543 (646)	total: 3m 53s	remaining: 2m

726:	learn: 0.4155401	test: 0.4234751	best: 0.4234751 (726)	total: 4m 27s	remaining: 1m 40s
727:	learn: 0.4155202	test: 0.4234624	best: 0.4234624 (727)	total: 4m 27s	remaining: 1m 39s
728:	learn: 0.4154971	test: 0.4234494	best: 0.4234494 (728)	total: 4m 28s	remaining: 1m 39s
729:	learn: 0.4154958	test: 0.4234464	best: 0.4234464 (729)	total: 4m 28s	remaining: 1m 39s
730:	learn: 0.4154952	test: 0.4234461	best: 0.4234461 (730)	total: 4m 28s	remaining: 1m 38s
731:	learn: 0.4154743	test: 0.4234433	best: 0.4234433 (731)	total: 4m 28s	remaining: 1m 38s
732:	learn: 0.4154681	test: 0.4234378	best: 0.4234378 (732)	total: 4m 29s	remaining: 1m 38s
733:	learn: 0.4154551	test: 0.4234288	best: 0.4234288 (733)	total: 4m 29s	remaining: 1m 37s
734:	learn: 0.4154429	test: 0.4234176	best: 0.4234176 (734)	total: 4m 29s	remaining: 1m 37s
735:	learn: 0.4154225	test: 0.4234100	best: 0.4234100 (735)	total: 4m 30s	remaining: 1m 36s
736:	learn: 0.4154057	test: 0.4234013	best: 0.4234013 (736)	total: 4m 30s	remain

816:	learn: 0.4134477	test: 0.4224812	best: 0.4224812 (816)	total: 5m 1s	remaining: 1m 7s
817:	learn: 0.4134243	test: 0.4224660	best: 0.4224660 (817)	total: 5m 1s	remaining: 1m 7s
818:	learn: 0.4134230	test: 0.4224648	best: 0.4224648 (818)	total: 5m 1s	remaining: 1m 6s
819:	learn: 0.4134030	test: 0.4224561	best: 0.4224561 (819)	total: 5m 2s	remaining: 1m 6s
820:	learn: 0.4133942	test: 0.4224473	best: 0.4224473 (820)	total: 5m 2s	remaining: 1m 5s
821:	learn: 0.4133786	test: 0.4224463	best: 0.4224463 (821)	total: 5m 2s	remaining: 1m 5s
822:	learn: 0.4133759	test: 0.4224454	best: 0.4224454 (822)	total: 5m 3s	remaining: 1m 5s
823:	learn: 0.4133285	test: 0.4224340	best: 0.4224340 (823)	total: 5m 3s	remaining: 1m 4s
824:	learn: 0.4133284	test: 0.4224339	best: 0.4224339 (824)	total: 5m 3s	remaining: 1m 4s
825:	learn: 0.4132870	test: 0.4224159	best: 0.4224159 (825)	total: 5m 4s	remaining: 1m 4s
826:	learn: 0.4132622	test: 0.4224037	best: 0.4224037 (826)	total: 5m 4s	remaining: 1m 3s
827:	learn

907:	learn: 0.4113544	test: 0.4216644	best: 0.4216644 (907)	total: 5m 36s	remaining: 34.1s
908:	learn: 0.4113526	test: 0.4216638	best: 0.4216638 (908)	total: 5m 36s	remaining: 33.7s
909:	learn: 0.4113509	test: 0.4216625	best: 0.4216625 (909)	total: 5m 36s	remaining: 33.3s
910:	learn: 0.4113034	test: 0.4216575	best: 0.4216575 (910)	total: 5m 37s	remaining: 33s
911:	learn: 0.4113028	test: 0.4216576	best: 0.4216575 (910)	total: 5m 37s	remaining: 32.6s
912:	learn: 0.4112814	test: 0.4216489	best: 0.4216489 (912)	total: 5m 38s	remaining: 32.2s
913:	learn: 0.4112370	test: 0.4216275	best: 0.4216275 (913)	total: 5m 38s	remaining: 31.8s
914:	learn: 0.4112335	test: 0.4216254	best: 0.4216254 (914)	total: 5m 38s	remaining: 31.4s
915:	learn: 0.4111630	test: 0.4215720	best: 0.4215720 (915)	total: 5m 38s	remaining: 31.1s
916:	learn: 0.4111451	test: 0.4215648	best: 0.4215648 (916)	total: 5m 39s	remaining: 30.7s
917:	learn: 0.4111132	test: 0.4215596	best: 0.4215596 (917)	total: 5m 39s	remaining: 30.4s
9

998:	learn: 0.4093341	test: 0.4208834	best: 0.4208834 (998)	total: 6m 8s	remaining: 369ms
999:	learn: 0.4092981	test: 0.4208687	best: 0.4208687 (999)	total: 6m 9s	remaining: 0us

bestTest = 0.4208686696
bestIteration = 999



In [70]:
prediction_proba = model.predict_proba(df_test)[:,1]

In [71]:
model.get_feature_importance(X_train,y_train,cat_features=categorical_features_indices)

[2.4904738937880913,
 0.9023635736328831,
 26.784972512305057,
 3.474099618979113,
 2.489420620267185,
 4.015760514421802,
 3.6905698914829443,
 6.242727532021042,
 3.7215939749033424,
 4.357634070441675,
 3.5198557625898284,
 6.269741184129969,
 5.607910307964413,
 1.9890194408451036,
 18.474678710283758,
 2.991705877361596,
 2.528727859987967,
 0.4487446545942221]

In [72]:
pd.Series(prediction_proba, 
          name='dep_delayed_15min').to_csv(f'{PATH}cat_encoded_flights_ods.csv', 
                                           index_label='id', header=True)

In [77]:
# This way we have randomness and are able to reproduce the behaviour within this cell.
np.random.seed(13)
from sklearn.model_selection import KFold

def impact_coding(data, feature, target='y'):
    '''
    In this implementation we get the values and the dictionary as two different steps.
    This is just because initially we were ignoring the dictionary as a result variable.
    
    In this implementation the KFolds use shuffling. If you want reproducibility the cv 
    could be moved to a parameter.
    '''
    n_folds = 10
    n_inner_folds = 5
    impact_coded = pd.Series()
    
    oof_default_mean = data[target].mean() # Gobal mean to use by default (you could further tune this)
    kf = KFold(n_splits=n_folds, shuffle=True)
    oof_mean_cv = pd.DataFrame()
    split = 0
    for infold, oof in kf.split(data[feature]):
            impact_coded_cv = pd.Series()
            kf_inner = KFold(n_splits=n_inner_folds, shuffle=True)
            inner_split = 0
            inner_oof_mean_cv = pd.DataFrame()
            oof_default_inner_mean = data.iloc[infold][target].mean()
            for infold_inner, oof_inner in kf_inner.split(data.iloc[infold]):
                # The mean to apply to the inner oof split (a 1/n_folds % based on the rest)
                oof_mean = data.iloc[infold_inner].groupby(by=feature)[target].mean()
                impact_coded_cv = impact_coded_cv.append(data.iloc[infold].apply(
                            lambda x: oof_mean[x[feature]]
                                      if x[feature] in oof_mean.index
                                      else oof_default_inner_mean
                            , axis=1))

                # Also populate mapping (this has all group -> mean for all inner CV folds)
                inner_oof_mean_cv = inner_oof_mean_cv.join(pd.DataFrame(oof_mean), rsuffix=inner_split, how='outer')
                inner_oof_mean_cv.fillna(value=oof_default_inner_mean, inplace=True)
                inner_split += 1

            # Also populate mapping
            oof_mean_cv = oof_mean_cv.join(pd.DataFrame(inner_oof_mean_cv), rsuffix=split, how='outer')
            oof_mean_cv.fillna(value=oof_default_mean, inplace=True)
            split += 1
            
            impact_coded = impact_coded.append(data.iloc[oof].apply(
                            lambda x: inner_oof_mean_cv.loc[x[feature]].mean()
                                      if x[feature] in inner_oof_mean_cv.index
                                      else oof_default_mean
                            , axis=1))

    return impact_coded, oof_mean_cv.mean(axis=1), oof_default_mean

In [81]:
features = df_raw.columns
numeric_features = []
categorical_features = []

for dtype, feature in zip(df_raw.dtypes, df_raw.columns):
    if dtype == object:
        #print(column)
        #print(train_data[column].describe())
        categorical_features.append(feature)
    else:
        numeric_features.append(feature)
categorical_features

['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest']

In [88]:
df_raw['target'] = y.map({'Y': 1, 'N': 0}).values

In [90]:
%%time
# Apply the encoding to training and test data, and preserve the mapping
impact_coding_map = {}
for f in categorical_features:
    print("Impact coding for {}".format(f))
    df_raw["impact_encoded_{}".format(f)], impact_coding_mapping, default_coding = impact_coding(df_raw, f,'target')
    impact_coding_map[f] = (impact_coding_mapping, default_coding)
    mapping, default_mean = impact_coding_map[f]
    df_test["impact_encoded_{}".format(f)] = df_test.apply(lambda x: mapping[x[f]]
                                                                         if x[f] in mapping
                                                                         else default_mean
                                                               , axis=1)

df_raw.to_csv(PATH + '\\impact_encoded_train.csv',index = False)
df_test.to_csv(PATH + '\\impact_encoded_test.csv',index = False)

Impact coding for Month
Impact coding for DayofMonth
Impact coding for DayOfWeek
Impact coding for UniqueCarrier
Impact coding for Origin
Impact coding for Dest
Wall time: 13min 44s


## Cheating Kaggle

In [6]:
!dir {PATH}

 Volume in drive D is Local Disk
 Volume Serial Number is B408-A348

 Directory of D:\Github\fastai\courses\ml1\AV_Mckin\ods

21-Apr-18  10:54 AM    <DIR>          .
21-Apr-18  10:54 AM    <DIR>          ..
18-Apr-18  08:21 AM    <DIR>          .ipynb_checkpoints
09-Apr-18  02:55 PM        30,203,258 add_test_dummies.csv
09-Apr-18  02:58 PM        35,412,316 add_test_dummiesv2.csv
09-Apr-18  02:55 PM        92,576,888 add_train_dummies.csv
09-Apr-18  02:58 PM       109,333,024 add_train_dummiesv2.csv
13-Apr-18  12:43 PM         2,659,732 cat_encoded_flights_ods.csv
07-Feb-18  08:21 PM         1,800,682 site_dic.pkl
21-Apr-18  10:09 AM         2,407,429 sub-210418.csv
21-Apr-18  10:09 AM         2,407,429 sub1.csv
17-Mar-18  11:25 PM        19,450,729 websites_test_sessions.csv
17-Mar-18  11:30 PM        60,526,801 websites_train_sessions.csv
              10 File(s)    356,778,288 bytes
               3 Dir(s)  183,789,621,248 bytes free


In [32]:
sub_df1 = pd.read_csv(f'{PATH}cat_encoded_flights_ods.csv');
sub_df2 = pd.read_csv(f'{PATH}cat_encoded_flights_ods.csv');

In [61]:
preds = (sub_df1.dep_delayed_15min * 1.3 + sub_df2.dep_delayed_15min*2.5)/4

In [62]:
max(preds)

0.90939239921497128

In [63]:
preds1 = np.where(preds>.80, .84, preds)

In [64]:
sub_df['dep_delayed_15min'] = preds

In [65]:
sub_df.to_csv(f'{PATH}modified_flights.csv',index=None,)